In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.pipeline import Pipeline as ImbPipeline 
from imblearn.over_sampling import SMOTE


In [2]:
df = pd.read_csv('/mnt/c/Users/AcerGamer/Downloads/Tuberculose/Projeto-Tuberculose/dados/arquivos_csv/dados_tuberculose_transformados.csv', low_memory=False)

In [3]:
# Ver tamanho e prévia
print(df.shape)
df.head(8)

(502271, 83)


,TP_NOT,ID_AGRAVO,DT_NOTIFIC,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,DT_DIAG,ANO_NASC,NU_IDADE_N,...,AGRAVDROGA,AGRAVTABAC,TEST_MOLEC,TEST_SENSI,ANT_RETRO,BAC_APOS_6,TRANSF,UF_TRANSF,MUN_TRANSF,NU_IDADE_ANOS
0,2,A169,1970-01-01 00:00:00.020200930,2020,15,150650,148.0,1970-01-01 00:00:00.020200930,2009.0,4011.0,...,2.0,2.0,5.0,NaN,NaN,NaN,2.0,15,15.0,NaN
1,2,A169,1970-01-01 00:00:00.020200710,2020,15,150530,149.0,1970-01-01 00:00:00.020200707,1973.0,4046.0,...,2.0,2.0,5.0,NaN,NaN,3,NaN,NaN,NaN,NaN
2,2,A169,1970-01-01 00:00:00.020201203,2020,22,220770,186.0,1970-01-01 00:00:00.020201119,1955.0,4065.0,...,2.0,2.0,5.0,NaN,NaN,NaN,2.0,22,22.0,NaN
3,2,A169,1970-01-01 00:00:00.020200210,2020,24,240920,141.0,1970-01-01 00:00:00.020200208,1960.0,4059.0,...,2.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,A169,1970-01-01 00:00:00.020210114,2021,43,430210,160.0,1970-01-01 00:00:00.020201226,1984.0,4035.0,...,2.0,1.0,5.0,NaN,NaN,3,NaN,NaN,NaN,NaN
5,2,A169,1970-01-01 00:00:00.020200811,2020,31,313240,145.0,1970-01-01 00:00:00.020200810,1968.0,4052.0,...,2.0,1.0,5.0,NaN,NaN,NaN,2.0,31,31.0,NaN
6,2,A169,1970-01-01 00:00:00.020200923,2020,41,411210,137.0,1970-01-01 00:00:00.020200923,1990.0,4029.0,...,2.0,2.0,5.0,NaN,NaN,NaN,2.0,41,41.0,NaN
7,2,A169,1970-01-01 00:00:00.020200804,2020,29,292350,140.0,1970-01-01 00:00:00.020200803,1972.0,4047.0,...,2.0,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Ver porcentagem de valores ausentes por coluna
porcentagem_nulos = df.isnull().mean() * 100

# Ver colunas com mais de 30% de nulos
colunas_para_remover = porcentagem_nulos[porcentagem_nulos > 30].index

# Remover essas colunas
df = df.drop(columns=colunas_para_remover)

print(f"Restaram {df.shape[1]} colunas após remoção de ausentes.")


Restaram 48 colunas após remoção de ausentes.


In [5]:
colunas_modelo = [
    'FORMA',            # variável alvo
    'CS_SEXO',
    'CS_RACA',
    'NU_IDADE_N',
    'HIV',
    'AGRAVAIDS',
    'AGRAVALCOO',
    'RAIOX_TORA',
    'BACILOSC_E',
    'CULTURA_ES',
    'NU_ANO'            # ano da notificação
]
df_modelo = df[colunas_modelo].copy()

### Engenharia de Features

In [7]:
# Extrai unidade (1º dígito) e valor (últimos 3)
df['unidade'] = df['NU_IDADE_N'] // 1000
df['valor'] = df['NU_IDADE_N'] % 1000

# Converte tudo para anos
def converter_idade(row):
    if row['unidade'] == 1:  # Horas
        return row['valor'] / (24 * 365)
    elif row['unidade'] == 2:  # Dias
        return row['valor'] / 365
    elif row['unidade'] == 3:  # Meses
        return row['valor'] / 12
    elif row['unidade'] == 4:  # Anos
        return row['valor']
    else:
        return None  # Ignorado ou inválido

df['Idade_Ano'] = df.apply(converter_idade, axis=1)

In [11]:
# 1. Criar Faixa Etária a partir da Idade
# Primeiro, vamos tratar possíveis NaNs em NU_IDADE_N para o pd.cut não falhar
mediana_idade = df_modelo['NU_IDADE_N'].median()
df_modelo['NU_IDADE_N'] = df_modelo['NU_IDADE_N'].fillna(mediana_idade)
bins = [0, 18, 35, 50, 65, 120]
labels = ['Jovem', 'Adulto Jovem', 'Adulto', 'Idoso', 'Muito Idoso']
df_modelo['faixa_etaria'] = pd.cut(df_modelo['NU_IDADE_N'], bins=bins, labels=labels, right=False)

In [7]:
# 2. Criar um contador de comorbidades/fatores de risco
# Assumindo que 1.0 ou 'Sim' representa a presença do agravo
df_modelo['total_comorbidades'] = (df_modelo['AGRAVAIDS'] == 1.0).astype(int) + \
                                  (df_modelo['AGRAVALCOO'] == 1.0).astype(int)

In [8]:
# 3. Criar feature de interação para HIV com AIDS
df_modelo['hiv_positivo_com_aids'] = ((df_modelo['HIV'] == 'Reagente') & (df_modelo['AGRAVAIDS'] == 1.0))

In [9]:
# Remover colunas originais que foram transformadas para evitar redundância
df_modelo = df_modelo.drop(columns=['NU_IDADE_N'])

In [9]:
print("Novas features criadas:")
print(df_modelo[['faixa_etaria', 'total_comorbidades', 'hiv_positivo_com_aids']].head(-20))

Novas features criadas:


KeyError: "None of [Index(['faixa_etaria', 'total_comorbidades', 'hiv_positivo_com_aids'], dtype='object')] are in the [columns]"

### --- Preparação dos Dados ---

In [11]:
df_modelo['FORMA'] = df_modelo['FORMA'].fillna(df_modelo['FORMA'].median())
y = df_modelo['FORMA'].astype(int)
X = df_modelo.drop('FORMA', axis=1)
y = df_modelo['FORMA'].astype(int)

In [12]:
# Identificar colunas numéricas e categóricas (APÓS a engenharia de features)
numeric_features = X.select_dtypes(include=['int64', 'float64', 'int32']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

#### --- Construção do Pipeline Avançado ---

In [13]:
#Criar o transformador para features numéricas
# Passo 1: Imputar com a mediana. Passo 2: Escalonar.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [14]:
# Criar o transformador para features categóricas
# Passo 1: Imputar com o valor mais frequente. Passo 2: One-Hot Encode.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
# Combinar os transformadores em um único pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [16]:
# Criar o pipeline final usando o ImbPipeline
# Este pipeline irá: 1. Pré-processar, 2. Selecionar Features, 3. Balancear, 4. Classificar
pipeline_final = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    # Seleciona features com base na importância de um modelo base
    ('selector', SelectFromModel(RandomForestClassifier(n_estimators=50, random_state=42))),
    # Aplica SMOTE para balancear as classes no conjunto de treino
    ('smote', SMOTE(random_state=42)),
    # O classificador final
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100))
])

#### --- Treinamento e Avaliação ---

In [17]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [18]:
# Treinar o pipeline completo
print("Iniciando o treinamento do pipeline completo...")
pipeline_final.fit(X_train, y_train)
print("Treinamento concluído.")

Iniciando o treinamento do pipeline completo...


/home/mrdalcol/jupyter_env/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['faixa_etaria']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


Treinamento concluído.


In [19]:
# Fazer predições no conjunto de teste
y_pred = pipeline_final.predict(X_test)

/home/mrdalcol/jupyter_env/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['faixa_etaria']. At least one non-missing value is needed for imputation with strategy='most_frequent'.
  warnings.warn(


In [20]:
# Avaliar o resultado
print("\nRelatório de Classificação Final:")
print(classification_report(y_test, y_pred))


Relatório de Classificação Final:
              precision    recall  f1-score   support

           1       0.97      0.58      0.73    128765
           2       0.37      0.61      0.46     17030
           3       0.05      0.49      0.09      4887

    accuracy                           0.58    150682
   macro avg       0.46      0.56      0.43    150682
weighted avg       0.87      0.58      0.68    150682



In [21]:
print("\nMatriz de Confusão Final:")
print(confusion_matrix(y_test, y_pred))


Matriz de Confusão Final:
[[74601 16771 37393]
 [  604 10360  6066]
 [ 1530   978  2379]]


### ignora daqui pra baixo

In [22]:
#df.info()

In [23]:
#X = df_modelo.drop('FORMA', axis=1)
#y = df_modelo['FORMA']

In [24]:
#categoricas = X.select_dtypes(include='object').columns.tolist()
#numericas = X.select_dtypes(include='number').columns.tolist()

In [25]:
# Pré-processamento
#preprocessador = ColumnTransformer(transformers=[
#    ('cat', OneHotEncoder(handle_unknown='ignore'), categoricas),
#    ('num', StandardScaler(), numericas)
#])

In [26]:
#pipeline = Pipeline(steps=[
#    ('preprocessador', preprocessador),
#    ('modelo', RandomForestClassifier(random_state=42, class_weight='balanced'))
#])
#model = RandomForestClassifier(class_weight='balanced', random_state=42)

In [27]:
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=0.3, random_state=42
#)

In [28]:
#colunas_categoricas = X.select_dtypes(include=['object']).columns
#colunas_numericas = X.select_dtypes(include=['int64', 'float64']).columns

#preprocessor = ColumnTransformer(
#    transformers=[
#        ('num', StandardScaler(), colunas_numericas),
#        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)
#    ]
#)

In [29]:
#X_train_encoded = preprocessor.fit_transform(X_train)
#X_test_encoded = preprocessor.transform(X_test)

In [30]:
#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X_train_encoded, y_train)
#  smote_nc = SMOTENC(categorical_features=[0, 2, 3], random_state=42)

In [31]:
#pipeline.fit(X_train, y_train)
#  model.fit(X_resampled, y_resampled) 


In [32]:
#y_pred = pipeline.predict(X_test)
#  y_pred = model.predict(X_test_encoded)
#print("Relatório de Classificação:")
#print(classification_report(y_test, y_pred))

#print("Matriz de Confusão:")
#print(confusion_matrix(y_test, y_pred))